# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the World of DeepLearning.AI

Step into the fascinating realm of DeepLearning.AI where innovation meets technology. Dive into the ocean of neural networks, algorithms, and data, revolutionizing industries worldwide. Whether you're a seasoned professional or a curious novice, DeepLearning.AI offers a wealth of resources to ignite your passion for AI. With online courses, cutting-edge research, and a vibrant community, the possibilities are endless. Join the movement that is shaping the future and embark on a journey of discovery with DeepLearning.AI. Let's unravel the mysteries of artificial intelligence together!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the World of DeepLearning.AI

Step into the fascinating realm of DeepLearning.AI where innovation meets technology. Dive into the ocean of neural networks, algorithms, and data, revolutionizing industries worldwide. Whether you're a seasoned professional or a curious novice, DeepLearning.AI offers a wealth of resources to ignite your passion for AI. With online courses, cutting-edge research, and a vibrant community, the possibilities are endless. Join the movement that is shaping the future and embark on a journey of discovery with DeepLearning.AI. Let's unravel the mysteries of artificial intelligence together!

--------------------------------------------------------------------------------
Critic (to Writer):


## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the World of DeepLearning.AI

Step into the fascinating realm of DeepLearning.AI where innovation meets technology. Dive into the ocean of neural networks, algorithms, and data, revolutionizing industries worldwide. Whether you're a seasoned professional or a curious novice, DeepLearning.AI offers a wealth of resources to ignite your passion for AI. With online courses, cutting-edge research, and a vibrant community, the possibilities are endless. Join the movement that is shaping the future and embark on a journey of discovery with DeepLearning.AI. Let's unravel the mysteries of artificial intelligence together!

--------------------------------------------------------------------------------

*******************


--------------------------------------------------------------------------------
Critic (to Writer):

Aggregated Feedback:
- Focus on Target Keywords, Optimize Meta Tags, Enhance Call-to-Action.
- Ensure that the content does not make any false or misleading claims, verify accuracy of claims regarding DeepLearning.AI resources, check for trademark or copyright infringement.
- Avoid exaggerated or misleading claims about effectiveness or outcomes, verify accuracy of statements about benefits or impact of DeepLearning.AI, promote in a transparent and truthful manner without creating unrealistic expectations.

Final Suggestions:
1. Incorporate target keywords, optimize meta tags, and enhance call-to-action to improve SEO.
2. Ensure that all claims made about DeepLearning.AI resources are accurate and avoid any false or misleading statements.
3. Check for potential trademark or copyright infringement to prevent legal issues.
4. Avoid any exaggerated or misleading claims about the effectiv

## Get the summary

In [ ]:
print(res.summary)